# RSA

First, an example by hand:

In [461]:
# Helpers
import random


def ext_gcd(a, b):
    x1, y1, x2, y2 = 1, 0, 0, 1

    while b:
        q = a // b
        a, b = b, a % b
        x1, x2 = x2, x1 - q * x2
        y1, y2 = y2, y1 - q * y2

    return a, x1, y1


# Key generation
p, q = 5, 7
n = p * q
print(f"p={p}, q={q}, n={n}")

phi = (p - 1) * (q - 1)
print(f"phi={phi}")

e = 5
a, d, _ = ext_gcd(e, phi)
assert a == 1, "e and phi must be coprime"
print(f"public key, e={e}")
print(f"secret key, d={d}")

# Encryption
m = 4
c = m ^ e
print(f"ciphertext, c={c}")

# Decryption
M = c ^ e
print(f"plaintext, M={M}")
assert m == M, "message m and plaintext M must match"

p=5, q=7, n=35
phi=24
public key, e=5
secret key, d=5
ciphertext, c=1
plaintext, M=4


In [464]:
order = 2**8


# Utilities


def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True


def rand_prime():
    while True:
        candidate = random.randint(2, order)
        if is_prime(candidate):
            return candidate


def rand_int(n):
    return random.randint(2, n)


def gcd(a, b):
    while b:
        a, b = b, a % b
    return a


def mod_inv(a, b):
    m0, x0, x1 = b, 0, 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1


def find_coprime(phi):
    e = rand_int(phi - 1)
    while gcd(e, phi) != 1:
        e = rand_int(phi - 1)
    return e


# Key generation


def key_gen():
    p, q = rand_prime(), rand_prime()
    n = p * q
    phi = (p - 1) * (q - 1)
    e = find_coprime(phi)
    d = mod_inv(e, phi)
    pk = (n, e)
    sk = (n, d)
    return pk, sk


# Encryption


def encrypt(msg, pk):
    n, e = pk
    ciphertext = pow(msg, e, n)
    return ciphertext


# Decryption


def decrypt(ciphertext, sk):
    n, d = sk
    plaintext = pow(ciphertext, d, n)
    return plaintext


pk, sk = key_gen()
msg = 123
ciphertext = encrypt(msg, pk)
plaintext = decrypt(ciphertext, sk)
assert msg == plaintext, "message must match plaintext"

print(f"msg={msg}")
print(f"pk={pk}")
print(f"sk={sk}")
print(f"ciphertext={ciphertext}")
print(f"plaintext={plaintext}")

msg=123
pk=(3749, 967)
sk=(3749, 1231)
ciphertext=3338
plaintext=123


## Attack Vector #1: Factorize $n$

In order to recover the private key $d$, we need to factorize $n$ into $p$ and $q$. Then, we simply compute $\phi(n) = (p - 1)(q - 1)$ and follow the key derivation steps.

For factorization, we use the [quadratic sieve](https://www.wikiwand.com/en/articles/Quadratic_sieve) algorithm.

Note that this only works for our selected, low bit security.


